In [58]:
import pandas as pd
pd.set_option("display.width",1000)

xls = pd.ExcelFile('customer_factory_14499.xlsx')

# 如果你想读取所有的工作表，可以遍历sheet_names
all_dfs = {sheet_name: pd.read_excel('customer_factory_14499.xlsx', sheet_name=sheet_name) for sheet_name in pd.ExcelFile('customer_factory_14499.xlsx').sheet_names}

In [59]:
visit_factory = all_dfs["Event - Visit Factory"]
factory_product_click = all_dfs["Event - Factory Product Click"]
product_view = all_dfs["Event - Product View"]


visit_factory['event_timestamp'] = pd.to_datetime(visit_factory['event_timestamp'], format='%Y/%m/%d/%H/%M/%S')
factory_product_click['event_timestamp'] = pd.to_datetime(factory_product_click['event_timestamp'], format='%Y/%m/%d/%H/%M/%S')


In [60]:
print(visit_factory.head(5))

  report_date     cookie_id  customer_id  ga_session_id     event_timestamp             event_name             event_value
0  2023-12-06  2.444236e+08        14499     1701895272 2023-12-06 12:41:56  Event - Visit Factory         Novel Nutrients
1  2023-12-06  2.444236e+08        14499     1701901610 2023-12-06 14:35:09  Event - Visit Factory  Qimei Industrial Group
2  2023-12-06  2.444236e+08        14499     1701901610 2023-12-06 14:40:32  Event - Visit Factory  Qimei Industrial Group
3  2023-12-06  2.444236e+08        14499     1701901610 2023-12-06 14:42:10  Event - Visit Factory  Qimei Industrial Group
4  2023-12-06  2.444236e+08        14499     1701904533 2023-12-06 15:17:57  Event - Visit Factory  Qimei Industrial Group


In [61]:
print(factory_product_click.head(5))

  report_date     cookie_id  customer_id  ga_session_id     event_timestamp                     event_name                                        event_value
0  2023-12-06  2.444236e+08        14499     1701895272 2023-12-06 12:42:30  Event - Factory Product Click  Ashwagandha Extract 10% Withanolides by Gravim...
1  2023-12-06  2.444236e+08        14499     1701901610 2023-12-06 14:39:55  Event - Factory Product Click                                  Allulose by Qimei
2  2023-12-06  2.444236e+08        14499     1701904533 2023-12-06 15:17:02  Event - Factory Product Click              Organic Red Beet Root Powder by Qimei
3  2023-12-06  2.444236e+08        14499     1701904533 2023-12-06 15:25:25  Event - Factory Product Click  Moringa Oleifera Leaf Extract 30% Saponnins by...
4  2023-12-06  2.444236e+08        14499     1701904533 2023-12-06 15:27:43  Event - Factory Product Click  Organic Tribulus Terrestris Powder Steam Treat...


In [62]:
print(product_view.head(5))

  report_date     cookie_id  customer_id  ga_session_id      event_timestamp            event_name                                        event_value
0  2023-12-06  2.444236e+08        14499     1701885460  2023/12/06/10/02/06  Event - Product View  Organic Tart Cherry Extract 10:1 by Hunan Huak...
1  2023-12-06  2.444236e+08        14499     1701885460  2023/12/06/10/11/27  Event - Product View  Epimedium Icariin 10% by Ingredients by Nature...
2  2023-12-06  2.444236e+08        14499     1701885460  2023/12/06/10/14/21  Event - Product View  Selenium Yeast Powder 2000ppm by Angel Yeast |...
3  2023-12-06  2.444236e+08        14499     1701889077  2023/12/06/10/58/31  Event - Product View  L-Selenomethionine 5000 DCP (Selenium 0.5%) by...
4  2023-12-06  2.444236e+08        14499     1701889077  2023/12/06/10/59/05  Event - Product View  L-Selenomethionine 5000 MD (Selenium 0.5%) by ...


In [63]:
# Correcting the format for datetime conversion
visit_factory['event_timestamp'] = pd.to_datetime(visit_factory['event_timestamp'], format='%Y/%m/%d/%H/%M/%S')
factory_product_click['event_timestamp'] = pd.to_datetime(factory_product_click['event_timestamp'], format='%Y/%m/%d/%H/%M/%S')

# Perform the operation again with corrected datetime format
cross_joined_df = factory_product_click.assign(key=1).merge(
    visit_factory.assign(key=1),
    on='key',
    suffixes=('_fp_click', '_visit')
).query(
    "customer_id_fp_click == customer_id_visit & "
    "ga_session_id_fp_click == ga_session_id_visit & "
    "event_timestamp_fp_click > event_timestamp_visit"
)

# Find the closest visit event for each product click event
closest_matches = cross_joined_df.groupby(['cookie_id_fp_click', 'event_timestamp_fp_click'], as_index=False).apply(
    lambda x: x.loc[(x['event_timestamp_fp_click'] - x['event_timestamp_visit']).idxmin()]
).reset_index(drop=True)

# Selecting relevant columns for clarity and removing duplicate columns from join
final_df = closest_matches[['report_date_fp_click', 'cookie_id_fp_click', 'customer_id_fp_click',
                            'ga_session_id_fp_click', 'event_timestamp_fp_click', 'event_name_fp_click',
                            'event_value_fp_click', 'event_timestamp_visit', 'event_value_visit']]

final_df = final_df.rename(columns={
    'report_date_fp_click': 'report_date',
    'cookie_id_fp_click': 'cookie_id',
    'customer_id_fp_click': 'customer_id',
    'ga_session_id_fp_click': 'ga_session_id',
    'event_timestamp_fp_click': 'event_timestamp_click',
    'event_name_fp_click': 'event_name',
    'event_value_fp_click': 'factory_product_click',
    'event_timestamp_visit': 'event_timestamp_visit',
    'event_value_visit': 'visit_factory'
})

# Drop the 'event_name' column
final_df.drop('event_name', axis=1, inplace=True)

# Adjust the order of columns
final_df = final_df[[
    'report_date', 
    'customer_id', 
    'cookie_id', 
    'ga_session_id',	
    'event_timestamp_visit',
    'visit_factory',
    'event_timestamp_click',
    'factory_product_click'
]]

result_df = visit_factory.merge(
    final_df,
    left_on=['customer_id', 'ga_session_id', 'event_timestamp'],
    right_on=['customer_id', 'ga_session_id', 'event_timestamp_visit'],
    how='left'
)

# Drop specified columns from the result DataFrame
result_df_dropped = result_df.drop(['report_date_y', 'cookie_id_y', 'event_timestamp_visit', 'visit_factory'], axis=1)

# Rename the 'event_value' column to 'visit_factory'
result_df_dropped = result_df_dropped.rename(columns={'event_value': 'visit_factory'})

# Save to CSV
result_df_dropped.to_csv("result_df.csv")

In [64]:
# 分割event_value列
product_view[['product_name', 'product_id']] = product_view['event_value'].str.split('\|\|\|', expand=True)

# 删除原始的event_value列
product_view.drop(columns=['event_value'], inplace=True)

distinct_products = product_view[['product_name', 'product_id']].drop_duplicates()
distinct_products['product_name'] = distinct_products['product_name'].str.strip()

print(distinct_products)

                                         product_name               product_id
0   Organic Tart Cherry Extract 10:1 by Hunan Huakang   108928-HNHUAKANG-11283
1      Epimedium Icariin 10% by Ingredients by Nature   106714-000-10481-10281
2        Selenium Yeast Powder 2000ppm by Angel Yeast        43910-ANGEL-11426
3   L-Selenomethionine 5000 DCP (Selenium 0.5%) by...               69090-STEL
4   L-Selenomethionine 5000 MD (Selenium 0.5%) by ...        69091-NOVEL-12874
5   L-Selenomethionine 5000 DCP (Selenium 0.5%) by...     108785-SALVICI-12685
8   Ashwagandha Extract 10% Withanolides by Gravim...       109684-NOVEL-12874
11                         Selenium Glycinate by Aobo        106824-AOBO-11080
12  L-Selenomethionine 5000 DCP (Selenium 0.5%) by...      69090-SALVICI-99999
13               Tribulus Extract 95% Saponins by TRG          18912-TRG-11248
14  Organic Epimedium Leaf Powder (BRC Certified F...        38809-QIMEI-11712
15                                  Allulose by Qime

In [65]:
# 左连接result_df_dropped和distinct_products
merged_df = pd.merge(result_df_dropped, distinct_products, how='left', left_on='factory_product_click', right_on='product_name')

# 删除重复的product_name列
merged_df.drop(columns=['product_name'], inplace=True)

merged_df.to_csv("merged_df.csv")